In [1]:
!nvidia-smi

Thu Apr  4 12:22:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080        On  | 00000000:04:00.0 Off |                  N/A |
|  0%   26C    P8               6W / 180W |    666MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os

# Set the CUDA_VISIBLE_DEVICES environment variable to the index of the GPU you want to use
os.environ["CUDA_VISIBLE_DEVICES"] = "4" 

In [3]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("stanfordnlp/imdb")

/home/hygumm/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_df = dataset["train"].to_pandas()
test_df = dataset["test"].to_pandas()

In [5]:
df = pd.concat([train_df, test_df], ignore_index=True)
print("Merged DataFrame:")

Merged DataFrame:


In [6]:
import pandas as pd

# Assuming merged_df is already defined
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the shuffled DataFrame
print("Shuffled DataFrame:")
print(df)

Shuffled DataFrame:
                                                    text  label
0      Forget what I said about Emeril. Rachael Ray i...      0
1      Former private eye-turned-security guard ditch...      0
2      Mann photographs the Alberta Rocky Mountains i...      0
3      Simply put: the movie is boring. Cliché upon c...      0
4      Now being a fan of sci fi, the trailer for thi...      1
...                                                  ...    ...
49995  The "documentary", and we use that term loosel...      0
49996  This outlandish Troma movie is actually a very...      1
49997  I found the film Don't Look In The Basement to...      1
49998  I have read the novel Reaper of Ben Mezrich a ...      0
49999  Went to see this finnish film and I've got to ...      1

[50000 rows x 2 columns]


In [7]:
df.to_csv('imdb.csv', header=True)

In [8]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")

In [9]:
import pandas as pd
# df = pd.read_csv("file1.csv")
# df.head()

In [10]:
# df.rename(columns = {'review':'text'}, inplace = True) 

In [ ]:
#df.drop('Unnamed: 0', inplace= True, axis=1)

In [ ]:
#df = df.sample(n=1000, replace=False)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [13]:
from transformers import BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def preprocess_data(df, tokenizer, max_len=128):
    input_ids = []
    attention_masks = []

    for text in df['text']:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(df['label'].values)

    return input_ids, attention_masks, labels

# Preprocess both the training and testing set
train_inputs, train_masks, train_labels = preprocess_data(train_df, tokenizer)
test_inputs, test_masks, test_labels = preprocess_data(test_df, tokenizer)

In [14]:
import torch
torch.cuda.empty_cache()

In [15]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16  # Adjust based on your hardware capabilities

# Create the DataLoader for the training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for the test set
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [16]:
from transformers import BertForSequenceClassification

# Load the BERT model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=2,  # Binary classification
    output_attentions=False,
    output_hidden_states=False,
)

# Specify the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [17]:
import torch
torch.cuda.empty_cache()

In [18]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=2e-5,  # Learning rate
                  eps=1e-8)  # Epsilon to prevent division by zero in optimizer

# Number of training epochs
epochs = 4

# Total number of training steps is the number of batches * number of epochs
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,  # Default value
                                            num_training_steps=total_steps)


2024-04-04 12:29:01.180457: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-04 12:29:01.492131: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-04 12:29:02.752391: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-04-04 12:29:02.752429: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missi

In [19]:
import numpy as np
from sklearn.metrics import f1_score
import time
import datetime

# Function to calculate the accuracy of predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Function to format elapsed times as hh:mm:ss
def format_time(elapsed):
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Training loop
for epoch_i in range(0, epochs):
    
    # Perform one full pass over the training set.
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes
    t0 = time.time()

    # Reset the total loss for this epoch
    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a backward pass
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch)
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        loss = outputs.loss

        # Accumulate the training loss over all of the batches
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients
        loss.backward()

        # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update the learning rate
        scheduler.step()

    # Calculate the average loss over all of the batches
    avg_train_loss = total_train_loss / len(train_dataloader)
    print("\n  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on our validation set

    print("\nRunning Validation...")

    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in test_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():        
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)
            
        loss = outputs.loss
        logits = outputs.logits

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(test_dataloader)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

torch.save(model.state_dict(), 'final_bert_imdb.pt')

print("Training complete!")

======== Epoch 1 / 4 ========
Training...
  Batch    40  of  2,500.    Elapsed: 0:00:13.
  Batch    80  of  2,500.    Elapsed: 0:00:25.
  Batch   120  of  2,500.    Elapsed: 0:00:38.
  Batch   160  of  2,500.    Elapsed: 0:00:50.
  Batch   200  of  2,500.    Elapsed: 0:01:02.
  Batch   240  of  2,500.    Elapsed: 0:01:15.
  Batch   280  of  2,500.    Elapsed: 0:01:27.
  Batch   320  of  2,500.    Elapsed: 0:01:39.
  Batch   360  of  2,500.    Elapsed: 0:01:51.
  Batch   400  of  2,500.    Elapsed: 0:02:04.
  Batch   440  of  2,500.    Elapsed: 0:02:16.
  Batch   480  of  2,500.    Elapsed: 0:02:28.
  Batch   520  of  2,500.    Elapsed: 0:02:41.
  Batch   560  of  2,500.    Elapsed: 0:02:53.
  Batch   600  of  2,500.    Elapsed: 0:03:05.
  Batch   640  of  2,500.    Elapsed: 0:03:18.
  Batch   680  of  2,500.    Elapsed: 0:03:30.
  Batch   720  of  2,500.    Elapsed: 0:03:42.
  Batch   760  of  2,500.    Elapsed: 0:03:55.
  Batch   800  of  2,500.    Elapsed: 0:04:07.
  Batch   840  of 

In [20]:
def predict_sentiment(text, model, tokenizer):
    model.eval()

    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        return_token_type_ids=False,
        return_attention_mask=True,
        truncation=True,
        return_tensors='pt'
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).cpu().numpy()
    return predictions[0]

In [21]:
# Example usage
text = "I hate the  movie I did not like it at all"
prediction = predict_sentiment(text, model, tokenizer)
print("Predicted sentiment:", "Positive" if prediction == 1 else "Negative")

Predicted sentiment: Negative


In [22]:
# Example usage
text = '''"Aaaaand The Star Buoy hits it out of the park yet again!

What a hilarious ride. Tillu is a true blue phenomenon in the realm of Telugu Cinema 💙 
And nobody can do justice to it like Siddu!
What energy, what charm ❤️

Tillu is not to be reviewed, questioned, or analyzed. He is simply meant to be loved, 
so gooo watch and enjoy the fun partyyy! The one-liners and Anupama(superbly written - stellar performance)
are the other standouts in this Siddu Jonnalagadda  bonanza 🎉 Don'tttt missss!"'''
prediction = predict_sentiment(text, model, tokenizer)
print("Predicted sentiment:", "Positive" if prediction == 1 else "Negative")

Predicted sentiment: Positive


In [23]:
text = "One time watch .. good rom com movie but not up to the expectations…SIDDHU AS TILLU ROCKS again"
prediction = predict_sentiment(text, model, tokenizer)
print("Predicted sentiment:", "Positive" if prediction == 1 else "Negative")

Predicted sentiment: Positive
